# SAR-GE

SAR-GE is a Synthetic Aperture Radar payload carrying satellite that aims to detect Greenhouse Emissions from pipelines across Canada.

In [30]:
using Unitful
using Dates
module usedUnits
    using Unitful
    @unit day "day" Day 24u"hr" false
    @unit b "b" Bit 1 true
end
Unitful.register(usedUnits);

## Orbit
The orbit was designed using STK's orbit wizard - very convenient.

In [40]:
# Orbital Parameters
P = 86_170.5u"s";
e = 0;
i = 121u"°"; # Retrograde (i > 90°)
ω = 0;
Ω = 50u"°";
ν = 30u"°";

# Earth Parameters
μ = 3.986_004_418e14u"m^3*s^-2";
R_terra = 6_378.136_6u"km"; # SME-SMAD equatorial
J₂ = 0.001_082_635_9; # SME-SMAD
(u"minute"(P) - u"minute"(Dates.Day(1)))/1u"a" # we'll call this a rounding error? sidereal error? uhhh hhhhhhh
u"minute"(P)

1436.175 minute

SSE pg. 81 states that at high altitudes (such as in GEO), the gravitational pull of Luna and Sol become significant, as well as radiation pressure from Sol.

For circuluar orbits ($e=0$) $a=r$ and $P=2\pi\sqrt{a^3/\mu}$

In [42]:
# Orbit Radius and altitude
r = u"km"(∛((P/(2π))^2*μ));
h = r - R_terra # matches STK
n = u"day^-1"((√(μ/r^3))) # revolutions per day
h

35788.12397940338 km

In [4]:
# Vis viva to find total energy per unit mass and velocity
ε = u"J/kg"(-μ/(2*r))
V_c = u"m/s"(√(μ/r))

3074.583864909837 m s^-1

In [17]:
# Average values of Third Body Interaction (SME-SMAD)
DΩ_luna = -0.003_38u"°/day^2" * (cos(i)/n) # to make the units work, per SME-SMAD)
DΩ_sol = -0.001_54u"°/day^2" * (cos(i)/n) # "
Dω_luna = 0.001_69u"°/day^2" * (4u"day" - 5 * sin(i)^2/n); # "
Dω_sol = 0.000_77u"°/day^2" * (4u"day" - 5 * sin(i)^2/n); # "

In [18]:
# Solar radiation pressure (SME-SMAD)
#aᵣ = -4.5e-6 A/m [m/s^2] where A is cross-sectional area of sat. wrt. sol in m^2, m is mass of sat. in kg.
# per SME-SMAD pg. 214, above 800km altitude the acceleration from sol becomes an important perturbing force, frequently dominant wrt. other perturbations.

In [22]:
# Let's neglect atmospheric drag for now at least, as we are in GEO